In [7]:
import pandas as pd
from io import BytesIO
import os
import glob
from urllib.request import urlopen
from zipfile import ZipFile
from urllib.parse import urlparse


In [2]:
# New format data is available from 09-2015 to 07-2021
years = list(range(2019,2022))
years

[2019, 2020, 2021]

In [3]:
months = list(range(5,8))
months

[5, 6, 7]

In [4]:
months_str =  [str(month).zfill(2) for month in months]
months_str

['05', '06', '07']

In [5]:
# Cycle through years and months lists and generate rangle of zipped csv file urls to read data from
# exclude entries prior to September 2015
for year in years:
    for month in months:
        month_str = str(month).zfill(2)
        if ((year == 2015) & (month < 9)) :
            print("--")
        else :
            url="https://s3.amazonaws.com/tripdata/JC-"+ str(year) + month_str + "-citibike-tripdata.csv.zip"
#             print(url)

In [6]:
# Use zipfile library to unzip the csv files
for year in years:
    for month in months:
        # create a string representation of the month count with leading zeroes
        month_str = str(month).zfill(2)
        
        # check for exeptions
        if ((year == 2015) & (month < 9)) :
            print("-- data before Sep-2015 is out of scope")
        elif ((year == 2021) & (month > 7)) :
            print("-- data after Jul-2021 is unavailable")
        elif ((year == 2017) & (month == 8)) :
            # compose a unique url string for Aug 2017, as file name was mislabeled to contain space 
            # instead of a dash character after the year-month combination
            url="https://s3.amazonaws.com/tripdata/JC-201708%20citibike-tripdata.csv.zip"
        else :
            # compose url string  
            url="https://s3.amazonaws.com/tripdata/JC-"+ str(year) + month_str + "-citibike-tripdata.csv.zip"
            
        try :
            # use urlopen to access each zipped file online
            with urlopen(url) as zipresp:

                # cycle through each zip file 
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    print("Parsing " + str(year) + "-" + month_str +":" + url)
                    # extract content of each zip file into data directory
                    zfile.extractall('./data/')
        except :
            print("unable to parse url for " + str(year) + "-" + month_str)

Parsing 2019-05:https://s3.amazonaws.com/tripdata/JC-201905-citibike-tripdata.csv.zip
Parsing 2019-06:https://s3.amazonaws.com/tripdata/JC-201906-citibike-tripdata.csv.zip
Parsing 2019-07:https://s3.amazonaws.com/tripdata/JC-201907-citibike-tripdata.csv.zip
Parsing 2020-05:https://s3.amazonaws.com/tripdata/JC-202005-citibike-tripdata.csv.zip
Parsing 2020-06:https://s3.amazonaws.com/tripdata/JC-202006-citibike-tripdata.csv.zip
Parsing 2020-07:https://s3.amazonaws.com/tripdata/JC-202007-citibike-tripdata.csv.zip
Parsing 2021-05:https://s3.amazonaws.com/tripdata/JC-202105-citibike-tripdata.csv.zip
Parsing 2021-06:https://s3.amazonaws.com/tripdata/JC-202106-citibike-tripdata.csv.zip
Parsing 2021-07:https://s3.amazonaws.com/tripdata/JC-202107-citibike-tripdata.csv.zip


In [8]:
# Now concatenate all the csv files that were extracted
# Go down one level into the 'data' folder
os.chdir("./data")

In [9]:
# Match the pattern ‘csv’ and save the list of file names in the all_filenames variable. 
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['JC-201905-citibike-tripdata.csv',
 'JC-201906-citibike-tripdata.csv',
 'JC-201907-citibike-tripdata.csv',
 'JC-202005-citibike-tripdata.csv',
 'JC-202006-citibike-tripdata.csv',
 'JC-202007-citibike-tripdata.csv',
 'JC-202105-citibike-tripdata.csv',
 'JC-202106-citibike-tripdata.csv',
 'JC-202107-citibike-tripdata.csv']

In [10]:
test1 = pd.read_csv("JC-201907-citibike-tripdata.csv")
test2 = pd.read_csv("JC-202107-citibike-tripdata.csv")

In [11]:
# Review data in the old format
for col in test1.columns:
    print(f'\"{col}\": ,')

"tripduration": ,
"starttime": ,
"stoptime": ,
"start station id": ,
"start station name": ,
"start station latitude": ,
"start station longitude": ,
"end station id": ,
"end station name": ,
"end station latitude": ,
"end station longitude": ,
"bikeid": ,
"usertype": ,
"birth year": ,
"gender": ,


In [13]:
# Review data in teh new format
for col in test2.columns:
    print(f'\"{col}\"')

"ride_id"
"rideable_type"
"started_at"
"ended_at"
"start_station_name"
"start_station_id"
"end_station_name"
"end_station_id"
"start_lat"
"start_lng"
"end_lat"
"end_lng"
"member_casual"


In [14]:
# Reconcile column naming by performing blanket rename operation.
# First create dictionary of old vs new column names
col_name_reconcile = {
#     "tripduration": ,
    "starttime": "started_at",
    "stoptime": "ended_at",
    "start station id": "start_station_id",
    "start station name": "start_station_name",
    "start station latitude": "start_lat",
    "start station longitude": "start_lng",
    "end station id": "end_station_id",
    "end station name": "end_station_name",
    "end station latitude": "end_lat",
    "end station longitude": "end_lng",
#     "bikeid": ,
    "member_casual": "usertype"
#     "birth year": ,
#     "gender": 
}

In [15]:
# test1.rename(columns=col_name_reconcile, inplace=True)
# test1

In [16]:
# Match the member_casual(old) with usertype(new) columns, including corresponding variable values
df2cat = []
for f in all_filenames:
    a = pd.read_csv(f);
    a.rename(columns=col_name_reconcile, inplace=True)
    if "member_casual" in a.columns:
        a.rename(columns={"member_casual":"usertype"}, inplace=True)
    a['usertype']=a['usertype'].replace({"member" : "Subscriber", "casual" : "Customer"})
    df2cat.append(a)
df2cat

[       tripduration                started_at                  ended_at  \
 0               311  2019-05-01 00:07:36.4670  2019-05-01 00:12:47.6350   
 1               538  2019-05-01 00:08:29.2840  2019-05-01 00:17:27.9410   
 2               178  2019-05-01 00:39:36.4750  2019-05-01 00:42:34.9640   
 3               317  2019-05-01 01:24:28.0220  2019-05-01 01:29:45.5730   
 4               354  2019-05-01 01:54:39.6660  2019-05-01 02:00:34.1750   
 ...             ...                       ...                       ...   
 36130           792  2019-05-31 23:51:00.1170  2019-06-01 00:04:12.4530   
 36131           779  2019-05-31 23:51:22.7840  2019-06-01 00:04:22.3990   
 36132           347  2019-05-31 23:55:38.3850  2019-06-01 00:01:26.2030   
 36133           552  2019-05-31 23:56:11.9740  2019-06-01 00:05:24.8810   
 36134           520  2019-05-31 23:56:25.0240  2019-06-01 00:05:05.4860   
 
        start_station_id  start_station_name  start_lat  start_lng  \
 0              

In [17]:
#combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv = pd.concat(df2cat)
combined_csv.usertype.unique()

array(['Subscriber', 'Customer'], dtype=object)

In [19]:
combined_csv[combined_csv['tripduration'].isna()]['ended_at'].sort_values()

22964    2021-05-01 00:07:11
22077    2021-05-01 00:13:55
42571    2021-05-01 00:26:00
42765    2021-05-01 00:26:00
33536    2021-05-01 00:41:51
                ...         
25123    2021-08-01 13:55:10
59766    2021-08-01 15:03:00
24538    2021-08-01 20:23:24
25246    2021-08-01 21:25:49
25279    2021-08-01 21:50:34
Name: ended_at, Length: 199020, dtype: object

In [20]:
#export to csv
combined_csv.to_csv( "JC-citibike-tripdata_combined.csv", index=False, encoding='utf-8-sig')

In [21]:
print(combined_csv.shape)

(419009, 17)


In [220]:
trans_all.to_csv("nd_trans_stations.csv", index=False, encoding='utf-8')